In [1]:
import pyspark.sql.types as typ
import pandas as pd

Cargar dataset

In [2]:
datos = pd.read_csv("/Users/sergio/Desktop/Trabajo proyecto2/allData.csv")

datosDF = sqlContext.createDataFrame(datos)
datosDF=datosDF.drop("Unnamed: 0")


In [3]:
print(datos.loc[[1457]])

      Unnamed: 0 Aire_acondicionado Altura_sótano  Ancho_de_calle  \
1457        1457                  Y            TA            75.0   

      Año_de_construcción  Año_de_reforma    Base  Baños_a_medias_por_encima_  \
1457                 1965            1965  CBlock                           1   

      Baños_a_medias_sótano  Baños_enteros_por_encima      ...       \
1457                    0.0                         1      ...        

      Tipo_de_calefacción Tipo_de_garaje Tipo_de_venta Tipo_de_vivienda  \
1457                 GasA         Attchd            WD             1Fam   

      Total_pies_cuadr_sótano Valla  Valor_otras_caract Vecindario Zona  \
1457                   1256.0  None                   0    Edwards   RL   

     PrecioDeVenta  
1457        147500  

[1 rows x 76 columns]


In [4]:
labels= datosDF.dtypes

In [5]:
def get_dummy(df,categoricalCols,continuousCols,labelCol):
    
    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col
    
    indexers = [ StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c)) for c in categoricalCols ]
    
    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol="{0}_encoded".format(indexer.getOutputCol())) for indexer in indexers ]
    
    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders] + continuousCols, outputCol="features")
    
    pipeline = Pipeline(stages=indexers + encoders + [assembler])
    
    model=pipeline.fit(df)
    data = model.transform(df)
    
    data = data.withColumn('label',col(labelCol))

    
    return data.select('features','label')

In [6]:
catcols = catcols = ['Aire_acondicionado','Altura_sótano','Base','Calidad_chimeneas','Calidad_cocina','Calidad_garaje','Calidad_materiales_exterior','Calidad_y_condición_de_la_calefacción','Calidad_y_condición_piscina','Callejón','Clasificación_sótano_zona1','Clasificación_sótano_zona2','Condición_1','Condición_2','Condición_garaje','Condición_materiales_exterior','Condición_sótano','Condición_venta','Estilo_tejado','Funcionalidad_casa','Geografía_del_terreno','Localización','Material_tejado','Otras_características','Pavimento','Pavimento_entrada_coche','Pendiente','Plantas','Primer_revestimiento_exterior','Puertecita_sótano','Relieve','Segundo_revestimiento_exterior','Sistema_eléctrico','Situación_garaje','Tipo_de_bloque','Tipo_de_calefacción','Tipo_de_garaje','Tipo_de_venta','Tipo_de_vivienda','Valla','Vecindario','Zona']


num_cols = ['Ancho_de_calle','Año_de_construcción','Año_de_reforma','Baños_a_medias_por_encima_','Baños_a_medias_sótano','Baños_enteros_por_encima','Baños_enteros_sótano','Calidad_materiales','Calidad_vivienda','Cantidad_bloques_pies_cuadrados','Chimeneas','Clasificación','Coches_garaje','Cocina_por_encima','Dormitorios_por_encima','Pes_cuadr_baja_calidad','Pes_cuadr_totales_(sin_garaje)','Pies_cuadr_cerramiento','Pies_cuadr_cubierta_madera','Pies_cuadr_piscina','Pies_cuadr_poche_abierto','Pies_cuadr_porcha_cerrado','Pies_cuadr_segundo_piso','Pies_cuadr_sótano_sin_acabar','Pies_cuadr_z1_sótano','Pies_cuadr_z2_sótano','Pies_cuadrados_terreno','Precio_mensual','PrecioAnual','ScreenPorch','Total_pies_cuadr_sótano','Valor_otras_caract']
labelCol = 'PrecioDeVenta'

data = get_dummy(datosDF,catcols,num_cols,labelCol)


In [7]:
data.show(5)

+--------------------+------+
|            features| label|
+--------------------+------+
|(253,[0,2,5,10,16...|208500|
|(253,[0,2,6,12,15...|181500|
|(253,[0,2,5,12,16...|223500|
|(253,[0,1,7,11,16...|140000|
|(253,[0,2,5,12,16...|250000|
+--------------------+------+
only showing top 5 rows



In [8]:
from pyspark.ml.feature import StringIndexer
# Index labels, adding metadata to the label column
labelIndexer = StringIndexer(inputCol='label', outputCol='indexedLabel').fit(data)
labelIndexer.transform(data).show(5, True)

+--------------------+------+------------+
|            features| label|indexedLabel|
+--------------------+------+------------+
|(253,[0,2,5,10,16...|208500|       329.0|
|(253,[0,2,6,12,15...|181500|       286.0|
|(253,[0,2,5,12,16...|223500|       212.0|
|(253,[0,1,7,11,16...|140000|         1.0|
|(253,[0,2,5,12,16...|250000|        23.0|
+--------------------+------+------------+
only showing top 5 rows



In [9]:
from pyspark.ml.feature import VectorIndexer
import pyspark.ml.classification as cl

logistic = cl.LogisticRegression(
    maxIter=10,
    regParam=0.01,
    labelCol='indexedLabel')
featureIndexer =VectorIndexer(inputCol="features", \
                              outputCol="indexedFeatures", \
                              maxCategories=4).fit(data)
featureIndexer.transform(data).show(5, True)

+--------------------+------+--------------------+
|            features| label|     indexedFeatures|
+--------------------+------+--------------------+
|(253,[0,2,5,10,16...|208500|(253,[0,2,5,10,16...|
|(253,[0,2,6,12,15...|181500|(253,[0,2,6,12,15...|
|(253,[0,2,5,12,16...|223500|(253,[0,2,5,12,16...|
|(253,[0,1,7,11,16...|140000|(253,[0,1,7,11,16...|
|(253,[0,2,5,12,16...|250000|(253,[0,2,5,12,16...|
+--------------------+------+--------------------+
only showing top 5 rows



In [24]:
from pyspark.sql.functions import lit

In [25]:
data = data.withColumn("Id", lit(0))

In [26]:
train = sqlContext.createDataFrame(data.head(1458), data.schema)

In [27]:
test=data.subtract(train)

In [28]:
(trainingData, testData) = train.randomSplit([0.7, 0.3])

In [29]:
from pyspark.ml.classification import LogisticRegression
logr = LogisticRegression(featuresCol='indexedFeatures', labelCol='indexedLabel')

In [30]:
from pyspark.ml.feature import IndexToString
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="SalePrice",
                              labels=labelIndexer.labels)

In [31]:
from pyspark.ml import Pipeline
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, logr,labelConverter])

In [32]:
# Train model. This also runs the indexers.
model = pipeline.fit(trainingData)

In [33]:
prediction = model.transform(test)

In [34]:
predicted=prediction.select("SalePrice")

In [35]:
predicted.show()


+---------+
|SalePrice|
+---------+
|    87000|
|   110000|
|   172400|
|   244600|
|   120500|
|   185900|
|   143000|
|    67000|
|   148000|
|   154000|
|   119000|
|   110000|
|   325624|
|   154000|
|   143000|
|   130500|
|   154000|
|   140000|
|   178900|
|   134500|
+---------+
only showing top 20 rows



In [36]:
predicted.toPandas().to_csv('mycsv.csv')

In [37]:
from pyspark.ml.classification import RandomForestClassifier


In [38]:
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

In [39]:

pipeline3 = Pipeline(stages=[labelIndexer, featureIndexer, rf,labelConverter])

In [41]:

model3 = pipeline3.fit(trainingData)

In [42]:
predictions3 = model3.transform(test)

In [45]:
predicted3=predictions3.select("Id","SalePrice")

In [46]:
predicted3.show()

+---+---------+
| Id|SalePrice|
+---+---------+
|  0|   140000|
|  0|   140000|
|  0|   140000|
|  0|   220000|
|  0|   140000|
|  0|   140000|
|  0|   140000|
|  0|   125000|
|  0|   140000|
|  0|   145000|
|  0|   140000|
|  0|   140000|
|  0|   320000|
|  0|   175000|
|  0|   133000|
|  0|   130000|
|  0|   139400|
|  0|   140000|
|  0|   263435|
|  0|   140000|
+---+---------+
only showing top 20 rows



In [47]:
predicted3.toPandas().to_csv('mycsv3.csv')

In [48]:
import pyspark.ml.tuning as tune

In [ ]:
grid= tune.ParamGridBuilder().addGrid(logistic.maxIter,[2,10,50]).addGrid()

In [ ]:
trainingSummary.accuracy

str(trainingSummary.accuracy)

In [ ]:


str(trainingSummary.weightedPrecision)


Arbol

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dTree = DecisionTreeClassifier(labelCol='indexedLabel', featuresCol='indexedFeatures')

In [ ]:
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="SalePrice",
                               labels=labelIndexer.labels)
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dTree,labelConverter])
# Train model.  This also runs the indexers.
model2 = pipeline.fit(trainingData)

In [ ]:
prediction2 = model2.transform(test)


In [ ]:
predicted2=prediction2.select("Id","SalePrice")

In [ ]:
predicted2.toPandas().to_csv('mycsv2.csv')

In [ ]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.util import MLUtils

In [ ]:
model3 = NaiveBayes.train(trainingData, 1.0)

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql import Row
categoryIndexer = StringIndexer(inputCol="category", outputCol="label")
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=10000)
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
categoryConverter = IndexToString(inputCol="prediction", outputCol="predCategory", labels=categoryIndexer.labels)
pipeline = Pipeline(stages=[categoryIndexer, tokenizer, hashingTF, nb, categoryConverter])

model3 = pipeline.fit(train_data)

In [ ]:
# Make predictions.
predictions = model1.transform(testDF)
# Select example rows to display. 
predictions.select("features","label","predictedLabel").show(5)

In [ ]:
lrModel = model2.stages[-2]
trainingSummary2 = lrModel

In [ ]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions) 
print("Test Error = %g" % (1.0 - accuracy))
rfModel = model.stages[-2] 
print(rfModel) # summary only

naive bayes